#### CASCODE COMMON-SOURCE (11)
### Common source amplifier with cascode transistor or cascode current mirror

* The cascode transistor M2 for the common source amplifier stages M1 is an impedance converter. 
  * The input impedance of the cascode transitor M2 is low $\frac{1}{g_m}$ while the output impedance is large ${g_m}{r_{cascode}}{r_o}$
  * The M1 is sized for the required $g_m$, $r_o$ and the M2 is sized for overall desired $r_{cascode}$ 
  * The operating points are chosen as per the desired bias current in the branch which could be set by another active load device not shown here. The biasing of the cascode device could be generated by another bias voltage source, but for the purpose of the block , it is defined at a level of $2*V_{DSAT}+V_{THN}$ of about 1V. Hence, specifications: 
    * $V_{BIAS}$ = 1V
    * $I_{BIAS}$ = 1uA
    * $g_{m, M1}$ = 10mS
    * $r_{o, M1}$ = $\frac{1}{g_{ds, M1}}$ = 100KOhms
    * $r_{cascode, M2}$ = $\frac{1}{g_{ds, M2}}$ = 50KOhms

  * To verify, overall small signal output impedance is ${10{e}^{-3}}*{100e^3}*{5e^3}$ = $5e^6$ Ohms. 
    * Overall small signla gain = ${g_{m, M1}}*({g_{m, M1}}*{r_{o, M1}}*{r_{cascode, M2}})$ = $5e^4\frac{V}{V}$ = 93.9dB
      * This is assuming the load device has an impedance much higher than the output impedance and CLM doesn't impact the intrinsic output impedances of the devices. 

In [2]:
import numpy as np
import pandas as pd
from pygmid import Lookup as lk

In [3]:
nfet_01v8_lvt = lk('../techsweep/simulation/nfet_01v8_lvt.mat')

#### Specifications

In [4]:
gm = 10e-3
ro = 100e3
rcascode = 50e3

#### Design Choices

In [32]:
l = 0.15
# Large gmid for input device for higher gain in weak inversion
# & small gmid for cascode device to keep in deep saturation
gm_id_in = np.array([20, 10])
gm_id_casc = np.array([6, 15])

#### Biasing

In [55]:
vin_bias = 0.3
vcasc_bias = 1

#### Sizing and Benchmarking

In [84]:
# Sizing the input active device for amplications
id = gm/gm_id_in
# Look up JD for the requried GM_ID and L.
jd = nfet_01v8_lvt.lookup('ID_W', GM_ID=gm_id_in, L=l, vgs_w=vin_bias); print(jd)
# Get W from the ID and JD
w = id/jd
# Get the gds of the input device from looking up gmid for the length and then multiplying it with W.
gds_in = nfet_01v8_lvt.lookup('GDS_W', GM_ID=gm_id_in, L=l, vgs_w=vin_bias)*w
## The output impedance is inverse of the conductance.
rout_in = 1/gds_in
VTH_nfet = nfet_01v8_lvt.lookup('VT', GM_ID=gm_id_in, L=l, vg_ws=vin_bias); print(f"VT: ",VTH_nfet)
VDSAT_in = 2/(gm_id_in)
VGS_in = nfet_01v8_lvt.lookup('VGS', GM_ID=gm_id_in, L=l, vgs_w=vin_bias); print(f"VGS: ",VGS_in)

# print(f" Checking the availability of the data from datafame: id", id, jd, w)
# Printing for one column
# df = pd.DataFrame([gm_id_in, id, jd],['gm_id', 'id', 'jd'], columns=['option1'])
df = pd.DataFrame([gm_id_in, id, jd, w, gds_in, rout_in, VDSAT_in],
                    ['gm_id', 'id', 'jd', 'w', 'gds', 'ROUT', "VDSAT"], 
                        columns=[f'Option1 : l = {l}', f'Option2: l = {l}'])
df

[3.07739276e-06 3.23179774e-05]
VT:  []
VGS:  []


,Option1 : l = 0.15,Option2: l = 0.15
gm_id,20.000000,10.000000
id,0.000500,0.001000
jd,0.000003,0.000032
w,162.475199,30.942530
gds,0.000565,0.000599
ROUT,1770.212256,1670.351713
VDSAT,0.100000,0.200000


### Sizing and Benchmarking sizes for cascode transistor
## Constraints: 
* ID is as requried for biasing the input device. 
* gm_id is to bias the device in deep saturation. 
* Get the W for desired length.

In [89]:
id_casc = id
print(f"ID_CASCODE - 2 values",id_casc)

gm_casc = nfet_01v8_lvt.lookup('GM_ID', ID=id_casc, L=l, vgs_w=vcasc_bias); #print(gm_casc)
# jd_casc = nfet_01v8_lvt.lookup('ID_W', GM_ID=gm_casc/id_casc, L=1, vgs_w=vcasc_bias)
# jd_casc = nfet_01v8_lvt.lookup('ID_W', ID=id_casc, L=l, vgs_w=vcasc_bias); print(f"JD_CASC - 2 values",jd_casc)
jd_casc = nfet_01v8_lvt.lookup('ID_W', GM_ID=gm_id_casc, L=l, vgs_w=vcasc_bias); #print(f"JD_CASC - 2 VALUES",jd_casc)
w_casc = id_casc/jd_casc; #print(f"Width ", w_casc)
# w = nfet_01v8_lvt.lookup('w', ID=id_casc, L=l, gds=1/(rcascode))
gds_casc =  nfet_01v8_lvt.lookup('GDS_W', GM_ID=gm_id_casc, L=l, vgs_w=vcasc_bias)
rout_casc = 1/gds_casc
VDSAT_casc = 2/(gm_id_casc)

df_casc = pd.DataFrame([gm_id_casc, id_casc, jd_casc, w_casc, gds_casc, rout_casc, VDSAT_casc],
                    ['gm_id', 'id', 'jd', 'w', 'gds', 'ROUT', "VDSAT"], 
                        columns=[f'Option1 : l = {l}', f'Option2: l = {l}']); df_casc



ID_CASCODE - 2 values [0.0005 0.001 ]


,Option1 : l = 0.15,Option2: l = 0.15
gm_id,6.000000,15.000000
id,0.000500,0.001000
jd,0.000071,0.000012
w,7.042123,80.456703
gds,0.000028,0.000011
ROUT,35425.048322,93843.994945
VDSAT,0.333333,0.133333


#### Spice Netlist

In [68]:
%%writefile ./sizing_cascoded_common_source.spice
** Cascoded common source

.include /foss/pdks/sky130A/libs.tech/ngspice/corners/tt.spice
.param mc_mm_switch=0
.param lx=0.15 wx=162.5 nfx=40 idx=0.5m
.param wx_casc=7.05
.save @m.xM1.msky130_fd_pr__nfet_01v8_lvt
.save @m.xM2.msky130_fd_pr__nfet_01v8_lvt

* D G S B
xM2 out vbias d 0 sky130_fd_pr__nfet_01v8_lvt L={lx} W={wx_casc} nf={nfx} ad='int((nf+1)/2) * W/nf * 0.29' as='int((nf+2)/2) * W/nf * 0.29' pd='2*int((nf+1)/2) * (W/nf + 0.29)'
+ ps='2*int((nf+2)/2) * (W/nf + 0.29)' nrd='0.29 / W' nrs='0.29 / W' sa=0 sb=0 sd=0 mult=1 m=1
xM1 d vin 0 0 sky130_fd_pr__nfet_01v8_lvt L={lx} W={wx} nf={nfx} ad='int((nf+1)/2) * W/nf * 0.29' as='int((nf+2)/2) * W/nf * 0.29' pd='2*int((nf+1)/2) * (W/nf + 0.29)'
+ ps='2*int((nf+2)/2) * (W/nf + 0.29)' nrd='0.29 / W' nrs='0.29 / W' sa=0 sb=0 sd=0 mult=1 m=1

vg  vin  0  0.3
vb  vbias  0  1
is  out  0  {idx}

.control
  op
  *show
  print @m.xM1.msky130_fd_pr__nfet_01v8_lvt[gm]
  print @m.xM2.msky130_fd_pr__nfet_01v8_lvt[gm]
  print @m.xM1.msky130_fd_pr__nfet_01v8_lvt[id]
  print @m.xM2.msky130_fd_pr__nfet_01v8_lvt[id]
  print @m.xM1.msky130_fd_pr__nfet_01v8_lvt[gm]/print @m.xM1.msky130_fd_pr__nfet_01v8_lvt[id]
  print @m.xM2.msky130_fd_pr__nfet_01v8_lvt[gm]/print @m.xM2.msky130_fd_pr__nfet_01v8_lvt[id]
  print @m.xM1.msky130_fd_pr__nfet_01v8_lvt[gds]
  print 1/@m.xM1.msky130_fd_pr__nfet_01v8_lvt[gds]
  print @m.xM2.msky130_fd_pr__nfet_01v8_lvt[gds]
  print 1/@m.xM2.msky130_fd_pr__nfet_01v8_lvt[gds]
  print @m.xM1.msky130_fd_pr__nfet_01v8_lvt[gm]/(@m.xM1.msky130_fd_pr__nfet_01v8_lvt[gds]*@m.xM2.msky130_fd_pr__nfet_01v8_lvt[gds])
  
.endc
.end

Overwriting ./sizing_cascoded_common_source.spice


* gm, gm_id , gds of M1 transistor closely matches that from the look up data. 
* gm_id of M2 transistor is lower than look up data. The gds is higher than from look up data. 
  * This maybe possible because the Source of M2 is not at ground but at the Drain of the M1 transistor. 
    * Get the voltage at the drain of M1 and use it to redefine the sizing of M2.   